In [ ]:

xar = [50]
lidarData = [50]


import sys
import os
import struct
import matplotlib.pyplot as plt
import time
import math

ax = plt.subplot( polar=True)
plt.ylim(0,2)

#ax.set_rlabel_position(22.5)
ax.set_rticks([0.5, 1, 1.5, 2])

#plt.ion()
ax.set_thetamax(180)
 
plt.show()


import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility
np.random.seed(19680801)

# Compute areas and colors
N = 150
r = 2 * np.random.rand(N)
theta = 2 * np.pi * np.random.rand(N)
area = 200 * r**2
colors = theta

fig = plt.figure()
ax = fig.add_subplot(projection='polar')

ax.set_rlabel_position(90)
ax.scatter(theta, r, c=colors, s=area, cmap='hsv', alpha=0.75)
plt.show()

fig = plt.figure()
ax = fig.add_subplot(projection='polar')
c = ax.scatter(theta, r, c=colors, s=area, cmap='hsv', alpha=0.75)
ax.set_rlabel_position(50)
ax.set_thetamin(45)
ax.set_thetamax(135)


plt.show()


import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
import pandas as pd
import yfinance as yf
%matplotlib inline



data = pd.DataFrame()
TickerDax = ['ALV.DE', 'BMW.DE','FME.DE','BAS.DE','SAP.DE','DBK.DE','BAYN.DE','ADS.DE','CON.DE']

for x in TickerDax:
    t = yf.Ticker(x)
    yf.pdr_override()
    data[x] = t.history(start='2017-01-01', end='2017-12-31')['Close']
    
r = 0
AFactor = len(data['ALV.DE'])-1
n_TickerDax = len(TickerDax)
Vec1 = np.linspace(1,1, n_TickerDax)

data_R = pd.DataFrame()
Mdata_R = pd.DataFrame()
Sigma = pd.DataFrame()
Sigma_diag = pd.DataFrame()

for x in TickerDax:
    data_R[x] = (data[x]/data[x].shift(1) -1)
    
Sigma = AFactor * data_R.cov()
InvSigma = inv(Sigma)
for x in TickerDax:
    Sigma_diag[x] = [Sigma[x][x]]
    Mdata_R[x] = [float(np.mean(data_R[x]))]
    
Mean = AFactor* Mdata_R.iloc[0]
STD = Sigma_diag.iloc[0]**.5

a = Vec1.T.dot(InvSigma).dot(Vec1)
b = Mean.T.dot(InvSigma).dot(Vec1)
sd_a = 1/np.sqrt(a)

m_a = b/a
m_w = np.sqrt((Mean - b/a * Vec1).T.dot(InvSigma).dot( Mean - b/a * Vec1))
m_r = np.sqrt((Mean - r * Vec1).T.dot(InvSigma).dot(Mean - r * Vec1))
    
range_inf = np.min(Mean) - 0.05
range_sup = np.max(Mean) + 0.35

z2 = np.linspace(range_inf, range_sup, 50)
z1 = pd.DataFrame()
z = pd.DataFrame()
zr = pd.DataFrame()
i = 0
for i in range(len(z2)):
    z1[i] = [np.sqrt( ((z2[i] - m_a)/m_w)**2 + sd_a**2)]
    z[i] = [(z2[i] - b/a)/m_w]
    zr[i] = [(z2[i] - r)/m_r]
z1 = z1.iloc[0]
z = z.iloc[0]
zr=zr.iloc[0]

#plotting
fig, ax = plt.subplots()
colors = np.random.rand(len(STD))
plt. scatter(STD, Mean, c=colors, alpha= 0.7)
for i in range(len(TickerDax)):
    ax.annotate(TickerDax[i], (STD[i] + 0.01, Mean[i]))

plt.plot(z1,z2, alpha = 1)
plt.plot(z, z2, alpha = 1)
plt.plot(zr,z2,alpha = 1)
plt.legend(['points' ,'Frontier', 'Asymptot', 'capital market line' ], loc = 2)
plt.xlabel( 'Annualised standard deviation')
plt.ylabel('Annualised return')
plt.grid(True)
plt.xlim(0,0.35)
plt.ylim(-0.05,0.5)
plt.show()

import yfinance as yf

msft = yf.Ticker("AAPL")
#msft.info
msft.history_metadata

stock = yf.Ticker("AAPL")                           
temp = pd.DataFrame(stock.dividends)
div = temp.loc[start:end]

msft.get_shares_full(start="2022-01-01", end=None)

msft.income_stmt
msft.quarterly_income_stmt

msft.balance_sheet
msft.quarterly_balance_sheet

msft.cashflow
msft.quarterly_cashflow

# show holders
msft.major_holders
msft.institutional_holders
msft.mutualfund_holders

# show news
msft.news

msft.options #option dates
opt = msft.option_chain('2025-12-19')

#inflation process
#ARIMA, ARMA, GARCH?

import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('historical_prices.csv').set_index('date')
market_cap_df = pd.read_csv('historical_market_caps.csv').set_index('date')

total_market_cap = market_cap_df.sum(axis=1)
mu_market_cap = market_cap_df.div(total_market_cap, axis=0)

# Calculate market volatility and use it to adjust p
volatility = total_market_cap.pct_change().rolling(window=252).std() # using 1 year rolling window
p = 1 - volatility # adjust accordingly

# Calculate portfolio weights for each day
portfolio_market_cap = pd.DataFrame(index=mu_market_cap.index, columns=mu_market_cap.columns)
for date in mu_market_cap.index:
mu_p = mu_market_cap.loc[date].pow(p[date])
portfolio_market_cap.loc[date] = mu_p / mu_p.sum()

returns = df.pct_change()
portfolio_returns = (returns * portfolio_market_cap.shift()).sum(axis=1)

metrics = {
 'Total return': (portfolio_returns + 1).prod() - 1,
 'Average return': portfolio_returns.mean(),
 'Standard deviation': portfolio_returns.std(),
 'Sharpe ratio': portfolio_returns.mean() / portfolio_returns.std(),
 'Maximum drawdown': (portfolio_returns.cummax() - portfolio_returns).max(),
}

print(metrics)

(portfolio_returns + 1).cumprod().plot()
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True)
plt.show()

#Libraries
import EnvisionRiskRaaS as RaaS
from datetime import date
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import numpy as np
import queuelib
import seaborn as sns

#EnvisionRisk login
RaaS.envrsk_auth_log_in("mark@brezina.dk", "Ink-wire1x")

#### Get the delta vector (simulated price change) ####
# The term '_raw' within the function name is indicative of a key feature: 
# the simulated price changes are denominated in the same currency as 
# the stock. This implies that the analysis maintains the original currency 
# perspective of the stock, allowing for more accurate and relevant 
# insights. It enhances ease of comprehension and direct applicability, 
# bringing us closer to the context of the actual trading environment.
use_date = date.today()
use_symbol = "AAPL.US"
api_response = RaaS.envrsk_instrument_delta_vector_raw(use_date,use_symbol)

# Leverage our sophisticated API to conduct an Expected-Shortfall computation, 
# specified at a 97.5% confidence level across a single-day span, utilizing a 
# point-in-time volatility scenario.
api_response_es = RaaS.envrsk_instrument_expected_shortfall_raw(date = use_date,symbols = use_symbol, signif_level = 0.975, volatility_id = "point_in_time")
expected_shortfall_estimate = api_response_es["Output"]["expected_shortfall"][0]

#manipulate the data for the density function
data = api_response["Output"]["PnL"]

#titles and captions
subtitle = "Profit/Loss distribution (one day) for " + use_symbol + " as seen from the " + str(use_date)
caption= "In the world of financial markets, price uncertainty is a key element that contributes to market risk - the risk of losses in positions arising \nfrom movements in market prices. Risk management aims to quantify this risk and devise strategies to mitigate potential losses."

#plot, titles, vertical line and more.
plt.suptitle("Price Uncertainty", fontsize=18)
plt.title(subtitle, fontsize=10, loc='left')
sns.histplot(data, kde=True)
plt.axvline(x = expected_shortfall_estimate, color = "#57575F", label = 'axvline - full height')
plt.text(expected_shortfall_estimate, 300, 'Expected-Shortfall (97.5%, 1 day)', ha='center', va='center',rotation='vertical', backgroundcolor='white')
plt.xlabel("Profit/Loss (in $)")
plt.ylabel("Density")
plt.yticks([])
plt.text(5,-120, caption, ha='center', size=7)
plt.show()


import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, date, timedelta, timezone


def sdata(x, y, z):
    # x is a stock symbol
    # y is a start date
    # z is an end date
    st = pd.DataFrame()
    t = yf.Ticker(x)
    st = t.history(start=y, end=z)['Close']
    st.index = pd.to_datetime(st.index).tz_localize(None)
    return st

# dates in correct format for yahoo finance.
today = date.today() 
today = datetime(today.year,today.month,today.day) #today
five = datetime(today.year-5,today.month,today.day) #five years ago


portfolio = pd.read_excel(open('list.xlsx', 'rb'), sheet_name='list')
portfolio = portfolio['Symbols'].tolist()
portfolio = portfolio[0:10]

MR = pd.DataFrame()

for y in portfolio:
    
    x = sdata(y,five,today)
    
    lst = []
    for i in range(len(x)):
        lste = np.zeros(200)
        for j in range(1,200):
            if i >= j:
                lste[j] = np.mean(x[i-j:i])
        lst.append(lste)

    lst = pd.DataFrame(lst)
    lst[0] = list(x)




    lste = np.zeros(200)
    for i in range(1,len(lst)):

        for j in range(1,200):

            if lst[j][i] > lst[0][i] and lst[j][i-1] < lst[0][i-1]:
                lste[j] = lste[j] + 1
            elif lst[j][i] < lst[0][i] and lst[j][i-1] > lst[0][i-1]:
                lste[j] = lste[j] + 1

            if i == (len(lst)-1):
                lste[j] = len(lst) / lste[j]
                   
    MR[y] = lste

mean = []
for i in range(len(MR)):
    mean.append(np.mean(MR.loc[i]))
MR['MEAN'] = mean
display(MR)
plt.plot(MR.index, MR['MEAN'])


MRE = MR['MEAN']
MRE.to_excel('MRE.xlsx', sheet_name='MRE')

def g(x):
    return 1+x**(2/3)

for i in range(25,200):
    if g(i)*0.95 < MRE[i] and g(i)*1.05 > MRE[i]:
        print(i, "within 5%")

        
import pyodbc
import pandas as pd
server = 'localhost'
database = 'FinanceTest' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
            "Server=localhost;"
            "Database=FinanceTest;"
            "Trusted_Connection=yes;")
cursor = cnxn.cursor()
datasql = pd.read_sql("SELECT TOP(10) * FROM dbo.Customers", cnxn)
datasql


from pathlib import Path
from pypdf import PdfReader
from pypdf import PdfWriter

# Get path to pdfs directory
#pdf_path = (Path(__file__).parent/"calc_exercises"/"calc_textbook.pdf")

# reader and writer perform operations
reader = PdfReader("E:\Jobsearch\Quant.pdf")
writer = PdfWriter()

for page in reader.pages:
    # Extract text from page
    text = page.extract_text().lower()
    
    # Find all occurances of key word
    if text.find('quant') > -1:
        writer.add_page(page)

# Write all pages to new pdf, saves in working dir
writer.write('test.pdf')


import math

#fizzbuzz solution 1

def solution(n: int):
    list = []
    for i in range(1,n):  
        if i%3 == 0 and i%5 == 0:
            list.append("FizzBuzz")
        elif i%3 == 0:
            list.append("Fizz")
        elif i%5 == 0:
            list.append("Buzz")
        else:
            list.append(str(i))
    return list

#fizzbuzz solution 2

def solution(n: int):
    list = []
    
    def is_prime(n: int):
        if n <= 3:
            return n > 1
        if n % 2 == 0 or n % 3 == 0:
            return False
        limit = int(math.sqrt(n))
        for i in range(5, limit+1, 6):
            if n % i == 0 or n % (i+2) == 0:
                return False
        return True
    
    for i in range(1,n):  
        if is_prime(i):
            list.append("Prime") 
        elif i%3 == 0 and i%5 == 0:
            list.append("FizzBuzz")
        elif i%3 == 0:
            list.append("Fizz")
        elif i%5 == 0:
            list.append("Buzz")
        else:
            list.append(str(i))
    return list


#fizzbuzz solution 3

def solution(n: int, pi_length: int):
    list = []
    def is_pi(n:int, pi_length: int):
        s = str(math.pi)[0:pi_length]
        if str(n) not in s:
            return False
        elif str(n) in s:
            return True
    
    def is_prime(n: int):
        if n <= 3:
            return n > 1
        if n % 2 == 0 or n % 3 == 0:
            return False
        limit = int(math.sqrt(n))
        for i in range(5, limit+1, 6):
            if n % i == 0 or n % (i+2) == 0:
                return False
        return True

    for i in range(1,n):
        if is_pi(i, pi_length):
            list.append("Pi")
        elif is_prime(i):
            list.append("Prime")    
        elif i%3 == 0 and i%5 == 0:
            list.append("FizzBuzz")
        elif i%3 == 0:
            list.append("Fizz")
        elif i%5 == 0:
            list.append("Buzz")
        else:
            list.append(str(i))
    return list